In [1]:
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

df = pd.read_json("/content/drive/MyDrive/PerCQA_JSON_Format.json", encoding = 'utf=8=sig')

df.head()

,QID,QCATEGORY,QDATE,QUSERID,QTYPE,QGOLD_YN,QUsername,QBody,QSubject,Comments
0,1550088,NaN,2018-01-01T12:14:00,9101,General,Not Applicable,sami1366,دوستان نامزدم (شوهرم)باهام قهره امروز تولدشه ...,قهر شوهرم,"[{'CID': '50758322', 'CUSERID': 4918, 'CGOLD':..."
1,1558609,NaN,2018-01-06T00:17:00,6856,General,Not Applicable,khanoomi72,من خودم حس میکنم اشتباهم گیر دادن بیخودی و غر ...,بزرگترین اشتباه دوران نامزدیتون چی بوده؟؟,"[{'CID': '51114707', 'CUSERID': 3935, 'CGOLD':..."
2,1587820,NaN,2018-01-22T13:23:00,9918,General,Not Applicable,lol,سالگرد ازدواجم نزدیکه چی بخرم. سالگرد دوممونه....,کمک سالگرد ازدواجم نزدیکه,"[{'CID': '52336355', 'CUSERID': 10255, 'CGOLD'..."
3,4414921,NaN,2020-05-23T02:06:00,81403,General,Not Applicable,sepide1400,دختر من چهارده ماهه است. آیا برای تشخیص پرانتز...,متخصص ارتوپدی برای درمان پرانتزی پا,"[{'CID': '138006316', 'CUSERID': 38437, 'CGOLD..."
4,3109668,NaN,2019-08-13T12:07:00,25700,General,Not Applicable,نسا۷۹,چه پمادی بزنم و از چیه اینجور شده . واضح نیس...,لپ پسرم قرمز دون دونه,"[{'CID': '100859930', 'CUSERID': 33391, 'CGOLD..."


In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("BAAI/bge-m3")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [4]:
sample_questions = df["QBody"].dropna().sample(2, random_state=42).tolist()
embeddings = model.encode(sample_questions)

for i, sent in enumerate(sample_questions):
    print(f"Original QBody: {sent}")
    print(f"Embedding shape: {embeddings[i].shape}")
    print("-" * 80)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Original QBody: بچم یک سال و یک ماهشه اما خوابش کم هست در شب هشت یا نه ساعت اما با داد و گریه هی بیدار میشه. در روزم دو تا یک ساعت میخوابه.
Embedding shape: (1024,)
--------------------------------------------------------------------------------
Original QBody: پدیا شور خیلی وقته گیر نمیارم شیراز. کسی اطلاع داره؟ . بقیه شهرها چی؟
Embedding shape: (1024,)
--------------------------------------------------------------------------------


In [5]:
!pip install --upgrade lancedb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.8/34.8 MB 5.6 MB/s eta 0:00:00


- `Install LanceDB and Set Up an Embedding Function`: First we installed the LanceDB Python library. Then using the TextEmbeddingFunction class (a class from LanceDB for convert text to embedding using our model), we can define a custom embedding function that uses the bge-m3 model to encode question texts. We only use the dense embeddings.

    - *Dense Embeddings*: Dense embeddings are fixed-length vectors where all dimensions contain real values. They are generated by modern language models like bge-m3 and capture the full semantic meaning of a sentence.

    - *Sparse Embeddings*: Sparse embeddings, like those from TF-IDF or BM25, represent text based on word frequency. Most values are zero, and each dimension corresponds to a specific word in the vocabulary.

In [6]:
from lancedb.embeddings import TextEmbeddingFunction
from lancedb.embeddings.registry import register
from sentence_transformers import SentenceTransformer
from functools import lru_cache

@register("bge-m3")
class BGEM3Embedding(TextEmbeddingFunction):
    name: str = "BAAI/bge-m3"

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._ndims = None

    def generate_embeddings(self, texts):
        return self._model().encode(texts, normalize_embeddings=True).tolist()

    def ndims(self):
        if self._ndims is None:
            self._ndims = len(self.generate_embeddings(["مثال"])[0])
        return self._ndims

    @lru_cache()
    def _model(self):
        return SentenceTransformer(self.name)

- `Define a Schema for our Vector Database`: Our schema must include at least qid (a unique identifier for each entry), qbody (the raw question text), and embedding (the vector representation of the qbody using our custom embedding function).

In [11]:
import lancedb
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import EmbeddingFunctionRegistry

db = lancedb.connect("/content/ninisite_db")

bge_fn = BGEM3Embedding.create()

In [12]:
questions_df = df[["QID", "QBody"]].dropna().rename(columns={"QID": "qid", "QBody": "qbody"})

- `Create and Populate a LanceDB Table`: Now we create a new table using our defined schema. Then load the Nini Site dataset (questions only, excluding comments) into this table. The embeddings must be generated automatically by LanceDB using the custom embedding function and should not compute the embeddings manually before inserting the data into the database.

In [13]:
class SemanticSchema(LanceModel):
    embedding: Vector(bge_fn.ndims()) = bge_fn.VectorField()
    qbody: str = bge_fn.SourceField()
    qid: int

semantic_table = db.create_table("semantic_table", schema=SemanticSchema, mode="overwrite")

semantic_table.add(questions_df)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


AddResult(version=2)

In [14]:
semantic_table.to_pandas()

,embedding,qbody,qid
0,"[0.043938152, 0.03664006, -0.010585843, -0.010...",دوستان نامزدم (شوهرم)باهام قهره امروز تولدشه ...,1550088
1,"[-0.03775334, -0.0014564429, -0.030918509, -0....",من خودم حس میکنم اشتباهم گیر دادن بیخودی و غر ...,1558609
2,"[-0.029951325, 0.067306384, -0.038232263, 0.03...",سالگرد ازدواجم نزدیکه چی بخرم. سالگرد دوممونه....,1587820
3,"[-0.031048281, 0.018678509, -0.044434678, 0.01...",دختر من چهارده ماهه است. آیا برای تشخیص پرانتز...,4414921
4,"[-0.017344024, 0.03248695, -0.061496735, 0.000...",چه پمادی بزنم و از چیه اینجور شده . واضح نیس...,3109668
...,...,...,...
985,"[-0.029458774, 0.066051155, -0.016618922, 0.00...",هدایایی که برای بله برون می برن تو شهرهای مختل...,1582862
986,"[-0.017698696, 0.025286904, -0.035241596, -0.0...",براي اولين بار كره ي بادوم زميني خريدم ولي نمي...,1790522
987,"[-0.0115784565, 0.057263996, -0.02634234, -0.0...",من ۹ماهه که عقد کردم . نامزدم ماهی۱۱۰۰حقوقشه ت...,1574016
988,"[-0.02902578, 0.03061474, -0.01091088, -0.0193...",بچه ها از کی به نبودن شیر عادت میکنن؟ ۳ روزه گ...,4171615


In [24]:
queries = [
    "بارداری",
    "چگونه زودتر باردار شوم؟",
    "آیا لکه بینی نشانه‌ی بارداری است؟",
    "راه‌های طبیعی برای افزایش شانس بارداری چیست؟",
    "تأخیر در پریود چه دلایلی می‌تواند داشته باشد؟",
    "آیا حالت تهوع صبحگاهی همیشه نشانه بارداری است؟"
]


- `Perform Semantic Search with LanceDB`: Now we can retrieve the top 5 semantically similar results for at least 5 different user queries using LanceDB’s search function. Also we evaluate whether the results are semantically relevant to the query manually.

In [25]:
for q in queries:
    print(f"\nQuery: {q}")
    results = (
        semantic_table.search(model.encode(q).astype("float32"))
        .limit(5)
        .select(["qbody"])
        .to_pandas()
    )
    display(results)



Query: بارداری


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


,qbody,_distance
0,امکان بارداری بایه قطره هم هست خواهشابگین....ی...,0.788552
1,"بتا 0,1 بارداريه؟",0.827267
2,یک دختر بیست ماهه دارم. باید هجدهم ماه پریود...,0.842454
3,تو بارداری میشه مشکل ذهنی و جسمی بچه رو تشخیص ...,0.867174
4,کم کاری تیرویید دارم هفته 4 بارداری متوجه شدم ...,0.881270



Query: چگونه زودتر باردار شوم؟


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


...

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


,qbody,_distance
0,یک دختر بیست ماهه دارم. باید هجدهم ماه پریود...,0.703874
1,۳ ماه زایمان کردم از دیشب لک دیدم امروز پریود ...,0.800898
2,من دو ماهه زایمان کردم. با همسرم رابطه داشتم. ...,0.825518
3,بچه من واکسن ۱۸ ماهگیش دو هفته تاخیر داره الان...,0.836586
4,دیروز موعدم بود نشدم تا الان .بنظرتو بی بی چک ...,0.841981



Query: آیا حالت تهوع صبحگاهی همیشه نشانه بارداری است؟


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


,qbody,_distance
0,یک دختر بیست ماهه دارم. باید هجدهم ماه پریود...,0.894121
1,"بتا 0,1 بارداريه؟",0.916941
2,همش دل درد و تهوع میگیرم دکترم 100 بار رفتم می...,0.927696
3,الان اصلا آمادگیش رو ندارم سنم پایینه شاغلم ا...,0.928996
4,من تازه متوجه شدم باردارم. سه روز قبل موعد پری...,0.934513


- `Implement Classical Full-Text Search using LanceDB`: Now we index (the process of creating a structured summary of your data to enable faster search) fields for full-text search and create a full-text index on qbody and perform previously 5 queries. Then we compare the results with those from semantic search.

    - *Which approach gives more relevant results?*
        - We can observe that ***semantic search*** gives more relevant and better results, which finds results based on meaning using vector embeddings, while full-text search relies on exact keyword matching. Semantic search handles synonyms and rephrased questions better, making it more accurate for user queries.

Semantic search ranks results based on the ***distance*** between vector embeddings, smaller distance means higher similarity in meaning. Full-text search uses a ***score*** based on keyword matches, higher score means more matched or important terms.

In [26]:
semantic_table.create_fts_index("qbody", use_tantivy=False, replace=True)

In [27]:
for q in queries:
    print(f"\nQuery: {q}")
    results = (
        semantic_table.search(q, query_type="fts")
        .limit(5)
        .select(["qbody"])
        .to_pandas()
    )
    display(results)


Query: بارداری


,qbody,_score
0,امکان بارداری بایه قطره هم هست خواهشابگین....ی...,6.254519
1,چجور میشه تو دوران عقد سفت و محکم از بارداری ج...,5.972324
2,تو بارداری میشه مشکل ذهنی و جسمی بچه رو تشخیص ...,5.124545
3,در ماه سوم بارداری هستم و دچار یبوست شدم. دکت...,5.124545
4,کم کاری تیرویید دارم هفته 4 بارداری متوجه شدم ...,4.933547



Query: چگونه زودتر باردار شوم؟


,qbody,_score
0,چه کسایی با ای وی اف باردار شدن چجوری؟ هزینش...,14.478218
1,ما همسايمون دكل ايرانسل نصب كردن روخونشون ميخو...,13.019798
2,خانم های باردار شماها درد ناف یا دور ناف نداری...,12.835098
3,دخترم یک سال و نیم‌شه. کم و بیش یه کلماتی میگه...,11.240505
4,موهاشو یه ماه میشه کوتاه کردیم. خیلی قیافه اش ...,10.224316



Query: آیا لکه بینی نشانه‌ی بارداری است؟


...

,qbody,_score
0,دخترم ۱۶ ماهشه. همه چی اش خوبه از ۱۱ ماهگی را...,37.052994
1,دخترم ۲۲ ماهشه . کلا از اول ترسو بود. الان دوس...,36.607037
2,چجور میشه تو دوران عقد سفت و محکم از بارداری ج...,30.495209
3,خانوما با بچه های ۱/۵ساله چه بازی هایی میکنید؟...,24.285959
4,مامانای عزیزی که بچه کوچیک دارن . لطفا بگین چن...,22.657698



Query: تأخیر در پریود چه دلایلی می‌تواند داشته باشد؟


,qbody,_score
0,۳ ماه زایمان کردم از دیشب لک دیدم امروز پریود ...,41.390118
1,دختر دو ساله بایددر چه حد رشد ذهنی داشته باشه ...,36.444187
2,دکتر مغز و اعصاب به پسرم برای تأخیر گفتارش نوت...,34.346413
3,می شناسید تو دور و برتون دخترهایی که والدینشون...,34.346413
4,سلام دوستان. ما ایشالا فردا قراره بریم خواستگا...,24.994709



Query: آیا حالت تهوع صبحگاهی همیشه نشانه بارداری است؟


,qbody,_score
0,پسرم سه ماهش تموم شده. یه هفته است وارد چهار ...,33.031322
1,دختر من چهارده ماهه است. آیا برای تشخیص پرانتز...,32.658054
2,من هفت سال پیش مبتلا شدم اما چند سال هست که ب...,31.936258
3,داشتم بالكن ميشستم پاك كننده ريخته بودم يهو د...,29.643198
4,من اولین بچم دختر بود حالت تهوع داشتم ولی با ق...,26.637655


### 3.7. What is Hybrid Search?
Hybrid Search is a combination of semantic (vector-based) and lexical (full-text) search techniques.

####Why is it better?
Lexical Search is faster and works well for direct keyword matches.

Semantic Search captures meaning and can retrieve results with different wording but similar meaning.

Hybrid Search combines the strengths of both methods to improve accuracy and coverage.

####Practical Example:
For a query like "How can I get pregnant faster?", the system can:

Use lexical search to find posts that contain the exact phrase.

Use semantic search to find posts that discuss related concepts like "increase fertility chances".

Then merge and rank the results to provide a more relevant and comprehensive output.

### 3.8. Evaluation Methods for Search Systems

In this task, we manually evaluated whether the results returned by the system were semantically relevant. However, in practice, search systems are commonly evaluated using objective metrics. Below are the most widely used metrics:

#### Precision@k
- **Definition**: The proportion of the top k results that are relevant to the query.
- **Formula**:  
  Precision@k
  $$
  \text{Precision@k} = \frac{\text{Number of relevant documents in top } k}{k}
  $$
- **Use Case**: Useful when users care most about seeing a few highly relevant results (e.g., top 5).

#### Recall@k
- **Definition**: The proportion of all relevant documents that are retrieved in the top k.
- **Formula**:  
  Recall@k
  $$
  \text{Recall@k} = \frac{\text{Number of relevant documents in top } k}{\text{Total number of relevant documents}}
  $$

- **Use Case**: Important when it’s critical to find as many relevant results as possible (e.g., legal, medical, academic retrieval).

#### NDCG (Normalized Discounted Cumulative Gain)
- **Definition**: Measures ranking quality by giving higher importance to relevant documents appearing earlier in the result list.
- **Formula**:  
  DCG@k (Discounted Cumulative Gain)
  $$
  \text{DCG@k} = \sum_{i=1}^{k} \frac{rel_i}{\log_2(i + 1)}
  $$

  NDCG@k (Normalized DCG)
  $$
  \text{NDCG@k} = \frac{\text{DCG@k}}{\text{IDCG@k}}
  $$

  where $rel_i$ is the relevance of the result at position $i$, and $IDCG@k$ is the ideal DCG (i.e., the best possible ranking).
- **Use Case**: Best suited for evaluating ranked search results, such as in web search engines.



In [29]:
from sentence_transformers import CrossEncoder

reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
#reranker = CrossEncoder("BAAI/bge-reranker-base")

In [30]:
queries = [
    "مدت زمان بارداری؟",
    "علائم اولیه بارداری چیست؟",
    "تغذیه مناسب در بارداری؟",
    "درمان ناباروری در زنان",
    "زمان تست بارداری؟"
]

all_semantic_results = {}

for q in queries:
    results = (
        semantic_table.search(model.encode(q).astype("float32"))
        .limit(5)
        .select(["qbody"])
        .to_pandas()
    )
    all_semantic_results[q] = results


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [31]:
def rerank_query_answers(query, answers_df):
    pairs = [(query, row["qbody"]) for _, row in answers_df.iterrows()]
    scores = reranker.predict(pairs)

    reranked_df = answers_df.copy()
    reranked_df["reranker_score"] = scores
    reranked_df = reranked_df.sort_values("reranker_score", ascending=False).reset_index(drop=True)
    return reranked_df


In [32]:
all_reranked_results = {}

for query in queries:
    reranked = rerank_query_answers(query, all_semantic_results[query])
    all_reranked_results[query] = reranked


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packa

In [33]:
for query in queries:
    print(f"\n Query: {query}")
    print("Before Reranking:")
    display(all_semantic_results[query][["qbody", "_distance"]])

    print("After Reranking:")
    display(all_reranked_results[query][["qbody", "reranker_score"]])



 Query: مدت زمان بارداری؟
Before Reranking:


,qbody,_distance
0,یک دختر بیست ماهه دارم. باید هجدهم ماه پریود...,0.751272
1,من دو ماهه زایمان کردم. با همسرم رابطه داشتم. ...,0.772195
2,"بتا 0,1 بارداريه؟",0.791829
3,کیا تجربه داشتن یا اطرافشون دیدن که بچه ۲۸هفته...,0.821409
4,از دوران بارداریم سرگیجه گرفتم تا الانم خوب نش...,0.842020


After Reranking:


,qbody,reranker_score
0,من دو ماهه زایمان کردم. با همسرم رابطه داشتم. ...,9.315115
1,از دوران بارداریم سرگیجه گرفتم تا الانم خوب نش...,9.234332
2,یک دختر بیست ماهه دارم. باید هجدهم ماه پریود...,8.917538
3,کیا تجربه داشتن یا اطرافشون دیدن که بچه ۲۸هفته...,8.152018
4,"بتا 0,1 بارداريه؟",4.473942



 Query: علائم اولیه بارداری چیست؟
Before Reranking:


...

,qbody,reranker_score
0,آیا تنبلی تخمدان ممکنه بعد مدتی خوب بشه؟ من دو...,8.957613
1,عفونت واژن قارچی دارم. خیلی حالم بده میشه بگید...,8.676273
2,امکان بارداری بایه قطره هم هست خواهشابگین....ی...,8.669673
3,لطفا دکتر زنان خوب در تهران معرفی کنید,8.395889
4,راهکار بدین برای عفونت مکرر واژن و لیزرهای مکرر,6.454348



 Query: زمان تست بارداری؟
Before Reranking:


,qbody,_distance
0,کسی میدونه هزینه آزمایش قبل ازدواج الان چقد هست؟,0.700091
1,من تازه متوجه شدم باردارم. سه روز قبل موعد پری...,0.707572
2,بچه ها سیگار تو آزمایشات قبل ازدواج مشخص میشه؟...,0.762836
3,دیروز موعدم بود نشدم تا الان .بنظرتو بی بی چک ...,0.768101
4,یک دختر بیست ماهه دارم. باید هجدهم ماه پریود...,0.785078


After Reranking:


,qbody,reranker_score
0,بچه ها سیگار تو آزمایشات قبل ازدواج مشخص میشه؟...,8.907625
1,من تازه متوجه شدم باردارم. سه روز قبل موعد پری...,8.753080
2,یک دختر بیست ماهه دارم. باید هجدهم ماه پریود...,8.596855
3,دیروز موعدم بود نشدم تا الان .بنظرتو بی بی چک ...,8.283745
4,کسی میدونه هزینه آزمایش قبل ازدواج الان چقد هست؟,7.992228


### Step 4: Answer Reranking with Cross-Encoder (Bonus +10 pts)

To improve the quality of retrieved answers, we apply a **Reranker** model that evaluates each (query, answer) pair with deeper contextual understanding.

#### Purpose of Reranking

While semantic search using embeddings provides a fast and meaningful initial retrieval, it does not deeply evaluate how well each answer matches the user's question. Rerankers are designed to refine this by using **cross-encoders** that consider both the query and the candidate answer together.

#### How Reranking Works

1. **Semantic Search** retrieves top-*k* candidates using vector similarity (fast, approximate).
2. **Reranker** scores each (query, candidate) pair using a Cross-Encoder model.
3. Results are re-ordered based on these scores to better reflect true relevance.


